[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahmouddraz/xai/blob/main/notebooks/cam/cam_solution.ipynb)

# Imports 

In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import cv2 
import matplotlib.pyplot as plt
import scipy as sp 
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load model 


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/XAI workshop/Models/cnn_model_cam.h5')

In [ ]:
model.summary()

## Create CAM model

In [ ]:
cam_model = tf.keras.models.Model(inputs=model.input, outputs=(model.layers[-3].output, model.layers[-1].output))

# Take some images for testing

In [ ]:
!wget -O cat1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat1.jpg
!wget -O cat2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat2.jpg
!wget -O catanddog.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/catanddog.jpg
!wget -O dog1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog1.jpg
!wget -O dog2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog2.jpg

## Prepare image and show resuts

In [ ]:
tf.__version__

In [ ]:
image = 'cat1'

In [ ]:
def prepare_and_show_image(image): 
  # read image 
  image = cv2.imread(image)

  # resize image to to match the model input 
  image = cv2.resize(image, (300, 300))

  # normalize image 
  image = image /255
  
  # add one more axis 
  image_tensor = np.expand_dims(image, axis=0)

  #make a prediction
  features, res = cam_model.predict(image_tensor)
  
  #get the class_Activation features
  class_activation_features = sp.ndimage.zoom(features[0], (300/33, 300/33, 1), order=2)

  # get the class activation weights 
  class_activation_weights = model.layers[-1].get_weights()[0][:,0]

  # calculate the activation map 
  cam_output = np.dot(class_activation_features, class_activation_weights)
  
  # print and visualize 
  print("DOG" if round(res[0][0]) else "CAT")
  fig = plt.figure(figsize=(10,10))
  plt.imshow(cam_output, cmap='jet', alpha=0.6)
  plt.imshow(tf.squeeze(image_tensor), alpha=0.3)



In [ ]:
prepare_and_show_image('cat1.jpg')
prepare_and_show_image('cat2.jpg')
prepare_and_show_image('dog1.jpg')
prepare_and_show_image('dog2.jpg')